In [1]:
import sys
sys.path.append('C:/Users/wjd__/qua-libs/Tutorials/intro-to-octave')
from qm import QuantumMachinesManager
from qm.qua import *
from qm.octave import *
from configuration_TWPA_240722 import *
from qm import SimulationConfig
import time
import numpy as np

# Specify where to store the outcome of the calibration (correction matrix, offsets...)
octave_config.set_calibration_db(os.getcwd())
qmm = QuantumMachinesManager(host=qop_ip, cluster_name='TWPA', octave=octave_config)
qm = qmm.open_qm(config)


2024-07-22 18:12:08,122 - qm - INFO     - Starting session: 5357c2bd-91c7-4667-b217-24c5f4718d7b
2024-07-22 18:12:11,398 - qm - INFO     - Octave "oct1" Health check passed, current temperature 58
2024-07-22 18:12:11,403 - qm - INFO     - Performing health check
2024-07-22 18:12:11,416 - qm - INFO     - Health check passed
2024-07-22 18:12:12,182 - qm - WARNING  - Open QM ended with warning 0: Opening a new Quantum Machine and closing Quantum Machines: 
2024-07-22 18:12:12,184 - qm - WARNING  - Open QM ended with warning 0: Quantum Machine qm-1721660934500 is canceling running job 1721296029361.


2024-07-22 18:28:09,245 - qm - ERROR    - Octave "oct1" lost monitor connection


In [2]:
config

{'version': 1,
 'controllers': {'con1': {'analog_outputs': {1: {'offset': 0.0},
    2: {'offset': 0.0}},
   'digital_outputs': {1: {}},
   'analog_inputs': {1: {'offset': 0.0}, 2: {'offset': 0.0}}}},
 'elements': {'pump': {'RF_inputs': {'port': ('oct1', 1)},
   'RF_outputs': {'port': ('oct1', 1)},
   'intermediate_frequency': 100000000.0,
   'operations': {'cw': 'const', 'readout': 'readout_pulse'},
   'digitalInputs': {'switch': {'port': ('con1', 1),
     'delay': 87,
     'buffer': 15}},
   'time_of_flight': 24,
   'smearing': 0}},
 'octaves': {'oct1': {'RF_outputs': {1: {'LO_frequency': 8100000000.0,
     'LO_source': 'internal',
     'output_mode': 'always_on',
     'gain': 0}},
   'connectivity': 'con1'}},
 'pulses': {'const': {'operation': 'control',
   'length': 1000,
   'waveforms': {'I': 'const_wf', 'Q': 'zero_wf'}},
  'readout_pulse': {'operation': 'measurement',
   'length': 1000,
   'waveforms': {'I': 'readout_wf', 'Q': 'zero_wf'},
   'integration_weights': {'cos': 'cosine_

In [3]:
with program() as pump:
    with infinite_loop_(): # cw 1000ns 짜리를 무한히 계속 한다는건가(zero latency between iterations)
            play("cw"*amp(1.5), element="pump") 

In [5]:
job = qm.execute(pump)
time.sleep(3)
# job.halt()

2024-07-22 18:12:28,495 - qm - INFO     - Sending program to QOP for compilation
2024-07-22 18:12:28,554 - qm - INFO     - Executing program


In [4]:
job.halt()

True

In [12]:
# for_(var, start, end_cond, step)
with program() as pump_sweep:
    a = declare(fixed)
    # with infinite_loop_():
    with for_(a,0.1,a<1,a+0.01): # -2<amplitude<1.99 (programmed due to memory?)
            play(pulse="cw"*amp(a), element="pump") 

In [3]:
with program() as pump_sweep:
    a = declare(fixed)
    with infinite_loop_():
        with for_(a,0.1,a<1,a+0.01): # -2<amplitude<1.99 (programmed due to memory?)
            play(pulse="cw"*amp(a), element="pump") 

In [3]:
calibration = True
elements=["pump"]
if calibration:
    print("-" * 37 + " Play before calibration")
    # Step 5.1: Connect RF1 and run these lines in order to see the uncalibrated signal first
    job = qm.execute(pump)
    time.sleep(10)  # The program will run for 10 seconds
    job.halt()
    # Step 5.2: Run this in order to calibrate
    for element in elements:
        print("-" * 37 + f" Calibrates {element}")
        qm.calibrate_element(element, {6e9: (50e6,)})  # can provide many IFs for specific LO
    # Step 5.3: Run these and look at the spectrum analyzer and check if you get 1 peak at LO+IF (i.e. 6.05GHz)
    print("-" * 37 + " Play after calibration")
    job = qm.execute(pump)
    time.sleep(30)  # The program will run for 30 seconds
    job.halt()


------------------------------------- Play before calibration
2024-07-22 16:22:43,522 - qm - INFO     - Sending program to QOP for compilation


2024-07-22 16:22:43,601 - qm - INFO     - Executing program
------------------------------------- Calibrates pump
2024-07-22 16:22:54,820 - qm - INFO     - Compiling program


c:\Users\wjd__\anaconda3\envs\qua\lib\site-packages\qm\octave\_calibration_analysis.py:124: RuntimeWarning: invalid value encountered in scalar divide
  x0_ = -p_[1] / (2 * p_[3])
c:\Users\wjd__\anaconda3\envs\qua\lib\site-packages\qm\octave\_calibration_analysis.py:125: RuntimeWarning: invalid value encountered in scalar divide
  y0_ = -p_[2] / (2 * p_[5])
c:\Users\wjd__\anaconda3\envs\qua\lib\site-packages\qm\octave\_calibration_analysis.py:208: RuntimeWarning: invalid value encountered in scalar divide
  _I2c, _IQc, _Q2c = _I2c / _n, _IQc / _n, _Q2c / _n
c:\Users\wjd__\anaconda3\envs\qua\lib\site-packages\qm\octave\_calibration_analysis.py:124: RuntimeWarning: invalid value encountered in scalar divide
  x0_ = -p_[1] / (2 * p_[3])
c:\Users\wjd__\anaconda3\envs\qua\lib\site-packages\qm\octave\_calibration_analysis.py:125: RuntimeWarning: invalid value encountered in scalar divide
  y0_ = -p_[2] / (2 * p_[5])
c:\Users\wjd__\anaconda3\envs\qua\lib\site-packages\qm\octave\_calibration_a

2024-07-22 16:23:00,431 - qm - WARNING  - At least one of the correction values are out of range. values should be between -2 and 2 - 2 ** (-16), got [nan, nan, nan, nan]. Not setting the correction matrix.
------------------------------------- Play after calibration
2024-07-22 16:23:00,440 - qm - INFO     - Sending program to QOP for compilation
2024-07-22 16:23:00,512 - qm - INFO     - Executing program


In [70]:
qm.octave.set_lo_frequency('pump',6.3e9)
qm.octave.set_rf_output_gain('pump',-20)

# power
## amp
4. amp(0.1)->13.13
1. amp(1)->-14
6. amp(5)-> -14.06
3. amp(10)->-17.77
5. amp(20)->-13.13
2. amp(100)->-13.15

## gain
-20 -> -53

0 -> -33

20 -> -13


In [7]:
pip freeze

anyio==4.4.0
asttokens @ file:///home/conda/feedstock_root/build_artifacts/asttokens_1698341106958/work
backcall @ file:///home/conda/feedstock_root/build_artifacts/backcall_1592338393461/work
betterproto==2.0.0b5
certifi==2024.7.4
colorama @ file:///home/conda/feedstock_root/build_artifacts/colorama_1666700638685/work
comm @ file:///home/conda/feedstock_root/build_artifacts/comm_1710320294760/work
contourpy==1.2.1
cycler==0.12.1
datadog-api-client==2.26.0
debugpy @ file:///C:/b/abs_c0y1fjipt2/croot/debugpy_1690906864587/work
decorator @ file:///home/conda/feedstock_root/build_artifacts/decorator_1641555617451/work
dependency-injector==4.41.0
deprecation==2.1.0
exceptiongroup==1.2.2
executing @ file:///home/conda/feedstock_root/build_artifacts/executing_1698579936712/work
fonttools==4.53.1
grpcio==1.64.1
grpclib==0.4.5
h11==0.14.0
h2==4.1.0
hpack==4.0.0
httpcore==0.16.3
httpx==0.23.3
hyperframe==6.0.1
idna==3.7
importlib_metadata @ file:///home/conda/feedstock_root/build_artifacts/impo

In [8]:
pip install qm-qua==1.1.7.1

   ---------------------------------------- 0.0/312.5 kB ? eta -:--:--
   -- ------------------------------------ 20.5/312.5 kB 330.3 kB/s eta 0:00:01
   -- ------------------------------------ 20.5/312.5 kB 330.3 kB/s eta 0:00:01
   -- ------------------------------------ 20.5/312.5 kB 330.3 kB/s eta 0:00:01
   ------- ------------------------------- 61.4/312.5 kB 328.2 kB/s eta 0:00:01
   ------- ------------------------------- 61.4/312.5 kB 328.2 kB/s eta 0:00:01
   ----------- --------------------------- 92.2/312.5 kB 350.1 kB/s eta 0:00:01
   --------------------- ---------------- 174.1/312.5 kB 583.1 kB/s eta 0:00:01
   -------------------------------------  307.2/312.5 kB 948.8 kB/s eta 0:00:01
   -------------------------------------- 312.5/312.5 kB 921.5 kB/s eta 0:00:00
  Attempting uninstall: qm-qua
    Found existing installation: qm-qua 1.1.7
    Uninstalling qm-qua-1.1.7:
      Successfully uninstalled qm-qua-1.1.7
Note: you may need to restart the kernel to use updated p

Reason for being yanked: Was supposed to be pre-release
